In [1]:
import pandas as pd
import csv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
path = '/Users/natapost/Documents/ultan_files/pl/2023_05_08_01/'
#file = '01_part1.tsv'

In [2]:
n = 0
with open(path + file) as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)
        n = n+1
        if n == 5:
            break

NameError: name 'csv' is not defined

In [5]:
def performance_calc(correct_column, window):
    performance = [float('nan')]*(window-1)
    l = len(correct_column)
    for i in range(l - window+1):
        p = sum(correct_column[i:(i+window)])/window
        performance.append(p)
    
    return performance

In [6]:
data = pd.DataFrame()
trials = []
go_list = []
relevant_modality = []
stages = []
space_pressed_list = []
correct_list = []
times = []
times_space = []
timestamps = []
timestamps_space = []
reaction_times = []

stage = 'training'
trial_on = False
space_pressed = False


log = '01_experiment_part2_2023_May_08_1051.log'
table = '01_experiment_part2_2023_May_08_1051.csv'
with open(path + log) as f:

    lines = f.readlines()
    for line in lines:
        if 'utcTime:' in line:
            #print(line)
            t1 = float(line.split(' \t')[0])
            t2 = float(line.split('utcTime: ')[1])
            dt = t2-t1
            #print(t1,t2)
            #print('dt_________',dt)
        
        if 'break' in line and 'DATA' in line:
            #print(line)
            pass
        
        if 'relevant_modality:' in line:
            r_mod = line.split('relevant_modality: ')[1][:3]
            #print(line)
        if 'next_stimuli' in line:
            #print(line)
            trial_n = line.split('trial_number=')[1].split(',')[0]
            go = line.split('trial_go=')[1].split(',')[0]
        if 'transition to' in line:
            #print(line)
            stage = line.split('transition to ')[1][:-1]

        if 'trial' in line and 'DATA' in line and 'next' not in line and 'order' not in line:
            #print(line)
            trial_on = True
            space_pressed = False
            time_space = float('nan')
            timestamp_space = float('nan')
            reaction_time = float('nan')

        if 'grating_trial: autoDraw = True' in line:
            if trial_on:
                time = float(line.split(' \t')[0])
                #print(line)
                timestamp = time + dt

        if 'Keypress: space' in line:
            #print(line)
            if trial_on:
                space_pressed = True
                time_space = float(line.split(' \t')[0])
                timestamp_space = time_space + dt
                reaction_time = time_space - time

        if 'response' in line:
            #print(line)

            correct = 0
            if go == str(space_pressed):
                correct = 1

            trial_on = False
            
            times.append(time)
            times_space.append(time_space)
            timestamps.append(timestamp)
            timestamps_space.append(timestamp_space)
            reaction_times.append(reaction_time)
            trials.append(trial_n)
            go_list.append(go)
            relevant_modality.append(r_mod)
            stages.append(stage)
            space_pressed_list.append(space_pressed)
            correct_list.append(correct)
            
        #if 'grating_trial: autoDraw = True' in line:
         #   print(line)
                
    
    data['time_stimuli_start'] = times
    data['time_space'] = times_space
    data['timestamp_stimuli_start'] = timestamps
    data['timestamp_key_pressed'] = timestamps_space
    data['RT'] = reaction_times
    data['relevant_modality'] = relevant_modality
    data['stage'] = stages
    data['trial_n'] = trials
    data['go'] = go_list
    data['key_pressed'] = space_pressed_list
    data['correct'] = correct_list
    data['performance_20wind'] = performance_calc(data.correct, 20)

In [7]:
data

,time_stimuli_start,time_space,timestamp_stimuli_start,timestamp_key_pressed,RT,relevant_modality,stage,trial_n,go,key_pressed,correct,performance_20wind
0,27.5241,28.3637,1.683532e+09,1.683532e+09,0.8396,AUD,staircase1,1,False,True,0,NaN
1,31.0380,31.4414,1.683532e+09,1.683532e+09,0.4034,AUD,staircase1,2,False,True,0,NaN
2,34.5570,NaN,1.683532e+09,NaN,NaN,AUD,staircase1,3,False,False,1,NaN
3,36.8081,37.6771,1.683532e+09,1.683532e+09,0.8690,AUD,staircase1,4,True,True,1,NaN
4,38.9283,39.6473,1.683532e+09,1.683532e+09,0.7190,AUD,staircase1,5,True,True,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
415,1086.5279,NaN,1.683533e+09,NaN,NaN,VIS,ids2,56,False,False,1,0.95
416,1088.7898,NaN,1.683533e+09,NaN,NaN,VIS,ids2,57,True,False,0,0.90
417,1091.0796,NaN,1.683533e+09,NaN,NaN,VIS,ids2,58,True,False,0,0.85
418,1093.1346,NaN,1.683533e+09,NaN,NaN,VIS,ids2,59,False,False,1,0.85


In [38]:

    
data['performance_20wind'] = performance_calc(data.correct, 20)
